In [1]:
import pandas as pd
import glob
import os

## Analyse Combined Data

In [2]:
curated_path = "../data/curated/merged_dataset/*.csv"
file_lst2 = []
for fname in glob.glob(curated_path):
    file_lst2.append(fname)
file_lst2
# combine all files in the list
merged_df = pd.concat([pd.read_csv(f) for f in file_lst2 ])
merged_df.dropna(inplace=True)
merged_df = merged_df[['year', 'sa2_2021', 'weekly_rent']]
merged_df = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
merged_df

,year,sa2_2021,weekly_rent
0,2013,201011001,297.545455
1,2013,201011002,303.958333
2,2013,201011005,319.411765
3,2013,201011006,375.333333
4,2013,201011007,360.625000
...,...,...,...
4879,2022,217031476,518.750000
4880,2022,217041477,414.069767
4881,2022,217041478,771.301370
4882,2022,217041479,437.025641


In [3]:
vic_sa2 = pd.read_csv('../data/curated/sa2_vic_2021.csv').iloc[:, 1:]
vic_sa2.rename({'SA2_CODE21': 'sa2_2021', 'SA2_NAME21': 'name_2021'}, axis=1, inplace=True)
vic_sa2

,name_2021,sa2_2021
0,Alfredton,201011001
1,Ballarat,201011002
2,Buninyong,201011005
3,Delacombe,201011006
4,Smythes Creek,201011007
...,...,...
519,Moyne - West,217041478
520,Warrnambool - North,217041479
521,Warrnambool - South,217041480
522,Migratory - Offshore - Shipping (Vic.),297979799


In [4]:
all_year_df = merged_df.merge(vic_sa2, how='left', left_on='sa2_2021', right_on='sa2_2021')
all_year_df

,year,sa2_2021,weekly_rent,name_2021
0,2013,201011001,297.545455,Alfredton
1,2013,201011002,303.958333,Ballarat
2,2013,201011005,319.411765,Buninyong
3,2013,201011006,375.333333,Delacombe
4,2013,201011007,360.625000,Smythes Creek
...,...,...,...,...
4879,2022,217031476,518.750000,Otway
4880,2022,217041477,414.069767,Moyne - East
4881,2022,217041478,771.301370,Moyne - West
4882,2022,217041479,437.025641,Warrnambool - North


## Geo Anlaysis

In [5]:
import geopandas as gpd
import json
import pandas as pd
with open('../data/curated/geo.json', 'r') as filename:
    geoJSON=json.load(filename)
geoJSON[:500]

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"SA2_NAME21": "Braidwood"}, "geometry": {"type": "Polygon", "coordinates": [[[149.76065447287993, -35.08346519409482], [149.76795447164443, -35.08445419474161], [149.76893249214802, -35.07943120479325], [149.78123347047955, -35.07791619586207], [149.78295448132192, -35.06941920595534], [149.77607047249325, -35.06843319534462], [149.77712948295027, -35.06367519540519], [149.78299149225955, -35.062153195909204'

In [6]:
#Preparing the geometries for neighbourhoods
neighbourhoods = gpd.read_file(geoJSON)

#The index of the json has to be the neighbourhood name
neighbourhoods.index = neighbourhoods['SA2_NAME21']
#Dropping useless columns for this application (it's all about saving memory)
neighbourhoods.drop(['SA2_NAME21','id'], axis=1, inplace=True)

#Choropleth mapbox accepts a json for the geometries of neighbourhoods.
neighbourhoods_json = json.loads(neighbourhoods.to_json())
neighbourhoods.head()

,geometry
SA2_NAME21,
Braidwood,"POLYGON ((149.76065 -35.08347, 149.76795 -35.0..."
Karabar,"POLYGON ((149.21921 -35.36010, 149.23878 -35.3..."
Queanbeyan,"POLYGON ((149.23149 -35.34224, 149.23155 -35.3..."
Queanbeyan - East,"POLYGON ((149.23155 -35.34221, 149.24149 -35.3..."
Queanbeyan West - Jerrabomberra,"POLYGON ((149.20647 -35.34586, 149.21921 -35.3..."


In [7]:
all_year_df = merged_df.merge(vic_sa2, how='left', left_on='sa2_2021', right_on='sa2_2021')


In [7]:
all_year_df.head()

,year,sa2_2021,weekly_rent,name_2021
0,2013,201011001,297.545455,Alfredton
1,2013,201011002,303.958333,Ballarat
2,2013,201011005,319.411765,Buninyong
3,2013,201011006,375.333333,Delacombe
4,2013,201011007,360.625000,Smythes Creek


In [ ]:

import folium
m = folium.Map(location=[-37.81, 144.96], tiles="Stamen Terrain", zoom_start=10, color='white')
svg_style = '<style>svg {background-color: rgb(255, 255, 255,0.5);}</style>'
m.get_root().header.add_child(folium.Element(svg_style))

c = folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
    data=all_year_df[all_year_df['year']==2022], 
    columns=['name_2021','weekly_rent'],
    key_on='properties.SA2_NAME21', 
    fill_color='PiYG', 
    nan_fill_color='black',
    legend_name='weekly rent by SA2',
)

c.add_to(m)
m

In [ ]:
max_count = all_year_df['weekly_rent'].max()

import plotly.express as px
fig = px.choropleth_mapbox(data_frame=all_year_df,
                           geojson=neighbourhoods,
                           locations=all_year_df['name_2021'],
                           color='weekly_rent',
                           center={'lat':-37.81, 'lon':144.96},
                           mapbox_style='open-street-map',
                           zoom=5,
                           color_continuous_scale='blues',
                           range_color=(0, 1000),
                           animation_frame='year',
                           width=800,
                           height=600)
fig.write_html('plow_map.html')
fig.show()